In [1]:
import polars as pl
from datetime import datetime, timedelta

pl.Config.set_fmt_str_lengths(200)
pl.Config.set_fmt_float("full")

polars.config.Config

Objective:

Determine what Holesky block composition is primarily made of.

In [2]:
# start and end date range
start_date = datetime(2024, 6, 15)
end_date = datetime(2024, 6, 19)


# list comprehension to generate endpoints
beacon_urls = [
    f"https://data.ethpandaops.io/xatu/holesky/databases/default/canonical_beacon_block_execution_transaction/{
        date.year}/{date.month}/{date.day}.parquet"
    for date in (
        start_date + timedelta(days=n) for n in range((end_date - start_date).days + 1)
    )
]

In [3]:
beacon_df = (pl.concat(
    [pl.scan_parquet(file) for file in beacon_urls]
)
)

In [4]:
beacon_txs_df = beacon_df.select(
    'slot',
    'slot_start_date_time',
    'hash',
    'from',
    'to',
    'type',
).collect()

In [7]:
beacon_table = beacon_txs_df.group_by('to').agg(
    pl.len().alias('tx_count'),
    pl.col('type').mean().round(0).alias('tx_type')
)

In [8]:
beacon_table

to,tx_count,tx_type
binary,u32,f64
"b""0x023A6b4C3f477E808021696a06C902E0c75024E8""",1,2
"b""0xbC6F18B7D8BabB9635ae9e78f45bb9a0cdd64437""",2,2
"b""0xEbC2DDff1767557Ec22434FB5aA8c1AA1501EA2a""",1,2
"b""0x3aEcde33258bB507E747e7bdFdbF2Fcb42E72fca""",1,0
"b""0x1d43181E0e6343297A6C9c5C1CAeEC7c3Dc4ba34""",1,2
…,…,…
"b""0x1dEd43cb119FBB9E5D6e5fdF9f9F32c946D5eb0E""",1,2
"b""0xb3f39e171E3EC65702fC71bdC385aaa4817FAE27""",1,2
"b""0xcc4761675Da0ec4BEe4a4451a15CdcBE8B0ae98d""",1,2


In [10]:
beacon_table.sort(by='tx_count', descending=True).head(10)

to,tx_count,tx_type
binary,u32,f64
"b""0x3f5dFD1522a84394390382a9801616BD530A9655""",379718,2
"b""0x7F82C801D1778fC42Df04c22f532C5B18bB3ba0F""",181043,1
"b""0xEA593b730d745FB5fE01B6d20e6603915252c6BF""",134233,2
"b""0x0888e9E350ae4ac703e1e78341B180A007C15105""",133231,1
"b""0xFf00000000000000000000000000000000042069""",104358,2
"b""0x79C9109b764609df928d16fC4a91e9081F7e87DB""",81015,2
"b""0xCF7CDa60aC783a2336d665082c7eFf1aA0E88055""",25560,2
"b""0x3E4511645086a6fabECbAf1c3eE152C067f0AedA""",25213,1
"b""0xc144Bf0FF0Ff560784a881024ccA077ebaa5b163""",24114,1


In [13]:
# show recently active blobs. Seems like only the top 2 are still active today
beacon_table.filter(pl.col('tx_type') == 3).sort(
    by='tx_count', descending=True)

to,tx_count,tx_type
binary,u32,f64
"b""0xd8c5c541D56F59d65CF775de928CCf4a47d4985C""",14619,3
"b""0xD3f681bD6B49887A48cC9C9953720903967E9DC0""",7520,3
"b""0xFf00000000000000000000000000000900294412""",546,3
"b""0xff00000000000000000000000000000000000112""",413,3
"b""0x0000000000000000000000000000000000000000""",189,3
"b""0xFf00000000000000000000000000000017000920""",71,3
"b""0xBAc1000000000000000000000000000000000000""",37,3
"b""0x76c5E742B9144FB274Ed61aF022C88138fa3F852""",1,3
